# Pen Holster
I want a holster for the stylus that accompanies this product: https://www.amazon.com/gp/product/B01ICPGN46/ref=ppx_yo_dt_b_asin_title_o03_s00?ie=UTF8&psc=1

I want the holster to be strongly magnetic, so as to be applied to fridges or doors.

In [1]:
from solid import *
from solid.utils import *
import viewscad
r = viewscad.Renderer(openscad_exec='C:\Program Files\OpenSCAD\openscad.exe')

rez = 30 #30 for lo-rez; 100 for finished product

## Stylus Model
I think it would be good to work on making a model of the stylus itself.  Then the holster can be designed around it.

In [2]:
# basic shaft is a cylinder 123.4mm long, 9.1mm wide, 5.13mm thick
def ellipticalCylinder(rx, ry, height, center = False):
    return scale([1, ry/rx, 1])(cylinder(h=height, r=rx, center=center))

shaft = ellipticalCylinder(rx=9.1/2, ry=5.13/2, height=123.4)
r.render(shaft)

In [3]:
def cone(r1, r2, height, center=False):
    return cylinder(h=height, r1 = r1, r2 = r2, center = center)

def ellipticalCone(rx1,rx2,ry,height,center=False):
    return scale([1,ry/rx2,1])(cone(r1=rx1, r2=rx2, height=height,center=center))

tip = ellipticalCone(rx1=2/2,rx2=9.1/2,ry=5.13/2,height=15.5)
pen = tip + up(15.5)(shaft)

r.render(pen)

## Holster
I want the holster to essentially be a soft triagle with space in the center for the pen

In [4]:
starter = hull()(sum(rotate([0,0,120*i])(right(2)(circle(4))) for i in range(3)))
holster_block = linear_extrude(100,scale=1.5)(starter) 
r.render(holster_block)

In [5]:
pen_nullspace = hull()(sum(rotate([0,0,120*i])(rotate([0,1.2,0])(pen)) for i in range(3)))
r.render(pen_nullspace)
pen_nullspace = hole()(up(3)(pen_nullspace))
holster = holster_block + pen_nullspace
r.render(holster)

In [6]:
def roundedBox(w,d,h,filletR=0.1,center=False):
    pcloud  = translate([filletR, filletR, filletR])(sphere(filletR))
    pcloud += translate([filletR, filletR, h-filletR])(sphere(filletR))
    pcloud += translate([filletR, d-filletR, filletR])(sphere(filletR))
    pcloud += translate([filletR, d-filletR, h-filletR])(sphere(filletR))
    pcloud += translate([w-filletR, filletR, filletR])(sphere(filletR))
    pcloud += translate([w-filletR, filletR, h-filletR])(sphere(filletR))
    pcloud += translate([w-filletR, d-filletR, filletR])(sphere(filletR))
    pcloud += translate([w-filletR, d-filletR, h-filletR])(sphere(filletR))
    
    box = hull()(pcloud)
    
    if center:
        box = translate([-w/2,-d/2,-h/2])(box)
    
    return box

r.render(roundedBox(1,1,1,center=False))


In [7]:
magnet_box = roundedBox(4,14,120,center=True)
magnet_box = up(60)(left(7)(magnet_box))
magnet_holster = hull()(magnet_box + holster_block) + pen_nullspace
magnet_holster = rotate([0,0,-90])(up(9)(rotate([0,-90,0])(magnet_holster)))
r.render(magnet_holster)

In [10]:
def magnet():
    mag_dia = 8 #mm
    mag_h   = 3 #mm
    xtra    = 0.4 #mm
    s = (mag_dia+2*xtra)/mag_dia

    mag = cylinder(d=mag_dia, h=mag_h)
    mag = scale([s,s,1])(mag)
    magnet = hole()(mag)
    return magnet


In [11]:
mag1 = up(0.4)(forward(90)(magnet()))
mag2 = up(0.4)(forward(10)(magnet()))
final_holster = magnet_holster + mag1 + mag2
r.render(final_holster, outfile="holster.stl")